In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [4]:
%%time
import warnings
warnings.filterwarnings('ignore')
train_df = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']
param = {
    'bagging_freq': 5,          
    'bagging_fraction': 0.331,   'boost_from_average':'false',   
    'boost': 'gbdt',             'feature_fraction': 0.0405,     'learning_rate': 0.0083,
    'max_depth': -1,             'metric':'auc',                'min_data_in_leaf': 80,     'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,            'num_threads': 4,              'tree_learner': 'serial',   'objective': 'binary',      'verbosity': 1
}
folds = StratifiedKFold(n_splits=15, shuffle=False, random_state=2319)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
    clf = lgb.train(param, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 4000)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = predictions
sub.to_csv("submission.csv", index=False)

Fold 0
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.924552	valid_1's auc: 0.898921
[10000]	training's auc: 0.939897	valid_1's auc: 0.902141
[15000]	training's auc: 0.952567	valid_1's auc: 0.902135
Early stopping, best iteration is:
[12135]	training's auc: 0.945611	valid_1's auc: 0.902544
Fold 1
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.925025	valid_1's auc: 0.891776
[10000]	training's auc: 0.940394	valid_1's auc: 0.894133
[15000]	training's auc: 0.952955	valid_1's auc: 0.894247
Early stopping, best iteration is:
[13393]	training's auc: 0.949155	valid_1's auc: 0.894569
Fold 2
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.924326	valid_1's auc: 0.901471
[10000]	training's auc: 0.939786	valid_1's auc: 0.903285
[15000]	training's auc: 0.952419	valid_1's auc: 0.9033
Early stopping, best iteration is:
[11670]	training's auc: 0.944299	valid_1's auc: 0.903758
Fol

<bound method Booster.feature_importance of <lightgbm.basic.Booster object at 0x7fb1992a9e10>>